In [2]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats
import sqlite3
import time

#Create db
conn = sqlite3.connect('financial_data.db')

# Create a cursor
cursor = conn.cursor()
# Create the first table
all_time_prices_query = '''
CREATE TABLE all_time_prices (
    Date INTEGER PRIMARY KEY,
    Open FLOAT,
    High FLOAT, 
    Low FLOAT, 
    Close FLOAT, 
    Volume FLOAT,
    Dividends FLOAT, 
    Stock_Splits FLOAT, 
    Ticker TEXT NOT NULL    
);
'''

cursor.execute(all_time_prices_query)

conn.commit()


In [9]:
#Create first table with all stock info from yfinance
tick = pd.read_csv("all_stocks")
all_time_cols = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock_Splits', 'Ticker']
all_time = pd.DataFrame(columns = all_time_cols)
count = 0 
for i in range(len(tick)): 
    start = time.time()    
    try:
        #Creating the df to be added to all_time_prices
        ticker = tick["0"][i]
        info = yf.Ticker(ticker).history(period='max')
        info.reset_index(inplace = True)
        info['Ticker'] = ticker
        # Append the first dataframe to the table
        all_time = pd.concat([all_time, info])
        #print(momentum_append)
    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    except AttributeError: 
        print("Attribute error found")
        continue
    stop = time.time()
    duration = stop-start
    print(duration)
    all_time['Stock_Splits'] = all_time['Stock Splits']
    all_time.drop(["Stock Splits"], axis = 1, inplace=True)
    break

#PUSH TO DATABASE
all_time.to_sql('all_time_prices', conn, if_exists='append', index=False)
# Close the connection
conn.close()

0.8589463233947754


IntegrityError: datatype mismatch

In [10]:
all_time

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Ticker
0,1983-09-30 00:00:00-04:00,0.000000,0.467834,0.450507,0.450507,18000,0.0,0.0,AOS
1,1983-10-03 00:00:00-04:00,0.000000,0.453395,0.447619,0.450507,18000,0.0,0.0,AOS
2,1983-10-04 00:00:00-04:00,0.000000,0.447619,0.444731,0.444731,21600,0.0,0.0,AOS
3,1983-10-05 00:00:00-04:00,0.000000,0.441843,0.392749,0.392749,127800,0.0,0.0,AOS
4,1983-10-06 00:00:00-04:00,0.000000,0.401413,0.389862,0.401413,118800,0.0,0.0,AOS
...,...,...,...,...,...,...,...,...,...
9926,2023-02-16 00:00:00-05:00,66.699997,68.190002,66.360001,67.720001,742000,0.0,0.0,AOS
9927,2023-02-17 00:00:00-05:00,67.480003,67.480003,65.470001,67.239998,1178700,0.0,0.0,AOS
9928,2023-02-21 00:00:00-05:00,66.589996,67.160004,65.290001,65.669998,1433400,0.0,0.0,AOS
9929,2023-02-22 00:00:00-05:00,65.919998,66.290001,65.150002,65.470001,626300,0.0,0.0,AOS
